In [157]:
import pandas as pd
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

data_path = "/run/media/kevin/Volume/OpenImages/"

In [158]:
class ImageDataset(Dataset):
    """Test Class"""
    
    def __init__(self, label_file, root_dir, transform=None):
        self.labels = pd.read_csv(label_file)
        self.root_dir = root_dir
        self.transform = transform
        
        # Make some adjustments
        self.labels.index = self.labels['ImageID']
        self.labels = self.labels[self.labels['Confidence'] == 1]
        self.label_names = np.sort(list(set(self.labels['LabelName'])))
        self.images = list(set(self.labels['ImageID']))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        """ Get an image"""
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        # Load image
        img_id = self.images[idx]
        img_path = os.path.join(self.root_dir, self.images[idx]+".jpg")
        image = Image.open(img_path)
        
        # Create label vector
        img_objects = np.array(self.labels.loc[img_id,]['LabelName'])
        image_label = np.array([1 if x in img_objects else 0 for x in self.label_names])
        
        # Apply transform
        if self.transform is not None:
            image = self.transform(image)
        
        image = np.asarray(image)
        sample = {'image': image, 'label': image_label}
        
        return sample
    
        

In [167]:
transform = transforms.Compose(
    [transforms.Scale((100, 100)),
     transforms.Grayscale(3),
     transforms.ToTensor()])

root_dir = os.path.join(data_path, "pics")
csv_path = os.path.join(data_path, "subset_train_annots.csv")
dataset = ImageDataset(label_file = csv_path, root_dir = root_dir, transform=transform)
sample = dataset[129]

## Model

In [168]:
n_classes = len(dataset.label_names)
print(f"{n_classes} classes")

601 classes


In [169]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(in_features=512, out_features=n_classes, bias=True)

In [170]:
# specify device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [171]:
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=6)
len(data_loader)

398079

In [ ]:
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=6)
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(data_loader):
       
        # get the inputs; data is a list of [inputs, labels]
        imgs = batch['image'].to(device)
        labels = batch['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(imgs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.000010
